<a href="https://colab.research.google.com/github/mwiewiorko/quantFinance/blob/master/1_model_process_with_trans_matrix_v_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
state=['1111','0111','1111','0111','1110','0110','1101','0101','1011','0011','1110','0110','1101','0101','1100','0100','1011','0011','1010','0010','1001','0001','1100','0100','1010','0010','1001','0001','1000','0000','m0','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','1000','0000','m0','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','0000','m11','deaktywacja']
state_next=['1111','1111','1110','1110','1101','1101','1011','1011','0111','0111','1100','1100','1010','1010','1001','1001','0110','0110','0101','0101','0011','0011','1000','1000','0100','0100','0010','0010','0001','0001','0001','0001','0001','0001','0001','0001','0001','0001','0001','0001','0001','0001','0000','0000','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','deaktywacja','deaktywacja','deaktywacja']
prob=['0,93','0,85','0,07','0,15','0,6','0,6','0,75','0,6','0,76','0,8','0,4','0,4','0,25','0,4','0,31','0,25','0,24','0,2','0,62','0,32','0,68','0,61','0,69','0,75','0,38','0,68','0,32','0,39','0,2','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,05','0,8','0,75','0,95','0,95','0,95','0,95','0,95','0,95','0,95','0,95','0,95','0,95','0,95','0,2','0,95','1']
#words = [w.replace('[br]', '<br />') for w in words] #https://stackoverflow.com/questions/3136689/find-and-replace-string-values-in-list
prob = [p.replace(',', '.') for p in prob]
prob = [float(p) for p in prob]


In [4]:
# inicial cases
n=10000
cases=pd.DataFrame(
    {
        'id':range(n),
        'balance_capital':np.random.uniform(1000,10000,n),
        'balance_interest':np.random.uniform(0,1000,n),
        'balance_other':np.random.uniform(0,500,n),
        'expected_payment':np.random.uniform(10,500,n),
        'state':np.random.choice(state,n)
    }
)
#cases


In [6]:
# Input data   
transitions_matrix = pd.DataFrame({'s_0': state, 's_1': state_next, 'p': prob}) 
#df.loc[:,"s_n+1"]
#transitions_matrix

In [25]:
#DEF of functions
### Def function to multiply matrixes in form of tables
def f_multiply_matrix(t_1,t_2,s_0='s_0',s_1='s_1'):
  import pandas as pd
  v_multiply = pd.merge(t_1,t_2,how="inner",left_on=s_0,right_on=s_1, suffixes=('_x', '_y') )
  v_multiply['p']=v_multiply['p_x']*v_multiply['p_y']
  v_multiply=v_multiply.groupby([s_0+'_y',s_1+'_x'],as_index=False).agg({'p': 'sum'})
  #v_multiply=v_multiply.rename(columns={[s_0+'_y': s_0, s_1+'_x': s_1})
  v_multiply.columns =v_multiply.columns.str.replace(r'_.$', '')
  return v_multiply

### If payment state
def f_if_payment_state(x):
  v_payment=0
  if x.isdigit():
    if x[-1]=='1':
      #print("ok")
      v_payment=1
  return v_payment
  
### Calc balance for 
def f_book_payments(capital,interest,other,payment):
  import numpy as np
  import pandas as pd
  # Asumption that capital>0,interest>0,other>0
  payment_other=payment
  payment_interest=np.maximum(payment_other-other, 0)
  other_new=np.maximum(other-payment_other,0)
  payment_capital=np.maximum(payment_interest-interest, 0)
  interest_new=np.maximum(interest-payment_interest,0)
  capital_new=np.maximum(capital-payment_capital,0) 
  payments_final=capital-capital_new+interest-interest_new+other-other_new
  return  pd.DataFrame({'capital': capital_new,'interest': interest_new,'other': other_new, 'payment': payments_final })

In [28]:
#TEST FUNCTIONS
######f_multiply_matrix
#f_multiply_matrix(transitions_matrix,transitions_matrix)
######f_if_payment_state
# x='010'
# print(x[-1])
# f_if_payment_state(x)
###### f_book_payments
test_c=pd.Series([1000,100,10])
test_i=pd.Series([0,100,0])
test_o=pd.Series([100,0,10])
test_p=pd.Series([500,110,100])
f_book_payments(test_c,test_i,test_o,test_p)

,capital,interest,other,payment
0,600,0,0,500
1,90,0,0,110
2,0,0,0,20


In [43]:
# calc expected payments
## Agregate cases

result=cases.groupby(['state'],as_index=False).sum().drop('id', 1)#agg({'balance_capital': 'sum'})
result["time"]=0
result["payment"]=0
print(result)

for i in range(1,3):
  print(i)
  actual_state=result[result.time.eq(i-1)]
  next_state=pd.merge(actual_state,transitions_matrix,how="inner",left_on='state',right_on='s_0', suffixes=('_x', '_y') ) 
  next_state.time=i
  next_state.state=next_state.s_1
  next_state.balance_capital=next_state.balance_capital*next_state.p
  next_state.balance_interest=next_state.balance_interest*next_state.p
  next_state.balance_other=next_state.balance_other*next_state.p
  next_state.expected_payment=next_state.expected_payment*next_state.p
  next_state["payment"]=next_state.state.map(f_if_payment_state)*next_state.expected_payment
  new_balance=f_book_payments(next_state.balance_capital,next_state.balance_interest,next_state.balance_other,next_state.payment)
  next_state.balance_capital=new_balance.capital
  next_state.balance_interest=new_balance.interest
  next_state.balance_other=new_balance.other
  next_state.payment=new_balance.payment
  # drop innecesery cols
  next_state=next_state.drop(['s_0','s_1'], inplace=True, axis=1)
  result=result.append(next_state)

print(result)
print(next_state)



          state  balance_capital  ...  time  payment
0          0000     2.774597e+06  ...     0        0
1          0001     1.830673e+06  ...     0        0
2          0010     1.937245e+06  ...     0        0
3          0011     1.809078e+06  ...     0        0
4          0100     1.854821e+06  ...     0        0
5          0101     1.922490e+06  ...     0        0
6          0110     1.807185e+06  ...     0        0
7          0111     1.951468e+06  ...     0        0
8          1000     1.813888e+06  ...     0        0
9          1001     2.056140e+06  ...     0        0
10         1010     1.886624e+06  ...     0        0
11         1011     2.087190e+06  ...     0        0
12         1100     1.804746e+06  ...     0        0
13         1101     1.867034e+06  ...     0        0
14         1110     1.979085e+06  ...     0        0
15         1111     1.774297e+06  ...     0        0
16  deaktywacja     9.752088e+05  ...     0        0
17           m0     1.817948e+06  ...     0   

In [44]:
# result=cases.groupby(['state'],as_index=False).sum().drop('id', 1)#agg({'balance_capital': 'sum'})
# result["time"]=0
# result
# actual_state=result[result.time.eq(0)]
# actual_state
# pom_chr='abcdefghijk'
# pom_chr[-1]
# pom_chr.isdigit()
# print(actual_state)
# actual_state.state.map(f_if_payment_state)
for i in range(1,3):
  print(i)

1
2
